In [1]:
import argparse
import cv2
import numpy as np
import os
import torch
import pdb

from utils import setup_seed, read_points, read_calib, read_label, \
    keep_bbox_from_image_range, keep_bbox_from_lidar_range, vis_pc, \
    vis_img_3d, bbox3d2corners_camera, points_camera2image, \
    bbox_camera2lidar

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def point_range_filter(pts, point_range=[0, -39.68, -3, 69.12, 39.68, 1]):
    '''
    data_dict: dict(pts, gt_bboxes_3d, gt_labels, gt_names, difficulty)
    point_range: [x1, y1, z1, x2, y2, z2]
    논문에 표기된 대로 car, pedstrian cyclist를 검출하는 포인터 범위
    '''
    flag_x_low = pts[:, 0] > point_range[0] # 0
    flag_y_low = pts[:, 1] > point_range[1] # -39.68
    flag_z_low = pts[:, 2] > point_range[2] # -3
    flag_x_high = pts[:, 0] < point_range[3] # 69.12
    flag_y_high = pts[:, 1] < point_range[4] # 39.68
    flag_z_high = pts[:, 2] < point_range[5] # 1
    keep_mask = flag_x_low & flag_y_low & flag_z_low & flag_x_high & flag_y_high & flag_z_high
    pts = pts[keep_mask]
    return pts 

In [3]:
import numpy as np
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
from model.anchors import Anchors, anchor_target, anchors2bboxes
from ops import Voxelization, nms_cuda
from utils import limit_period


class PillarLayer(nn.Module):
    def __init__(self, voxel_size, point_cloud_range, max_num_points, max_voxels):
        super().__init__()
        self.voxel_layer = Voxelization(voxel_size=voxel_size,
                                        point_cloud_range=point_cloud_range,
                                        max_num_points=max_num_points,
                                        max_voxels=max_voxels)

    @torch.no_grad()
    def forward(self, batched_pts):
        '''
        batched_pts: list[tensor], len(batched_pts) = bs
        return: 
               pillars: (p1 + p2 + ... + pb, num_points, c), 
               coors_batch: (p1 + p2 + ... + pb, 1 + 3), 
               num_points_per_pillar: (p1 + p2 + ... + pb, ), (b: batch size)
        '''
        pillars, coors, npoints_per_pillar = [], [], []
        for i, pts in enumerate(batched_pts):
            voxels_out, coors_out, num_points_per_voxel_out = self.voxel_layer(pts) 
            # voxels_out: (max_voxel, num_points, c), coors_out: (max_voxel, 3)
            # num_points_per_voxel_out: (max_voxel, )
            pillars.append(voxels_out)
            coors.append(coors_out.long())
            npoints_per_pillar.append(num_points_per_voxel_out)
        
        pillars = torch.cat(pillars, dim=0) # (p1 + p2 + ... + pb, num_points, c)
        npoints_per_pillar = torch.cat(npoints_per_pillar, dim=0) # (p1 + p2 + ... + pb, )
        coors_batch = []
        for i, cur_coors in enumerate(coors):
            coors_batch.append(F.pad(cur_coors, (1, 0), value=i))
        coors_batch = torch.cat(coors_batch, dim=0) # (p1 + p2 + ... + pb, 1 + 3)

        return pillars, coors, npoints_per_pillar


In [9]:
pc_folder_path = "dataset/testing/velodyne/"
pc_file_list = os.listdir(pc_folder_path)
pc = read_points(pc_folder_path + "/" + pc_file_list[0])

pc = point_range_filter(pc)
# pc_torch = torch.from_numpy(pc)

In [10]:
pc.shape

(63339, 4)

In [13]:
if __name__ == "__main__":

    np.random.seed(22)

    device = torch.device("cuda:0")

    nclasses=3
    voxel_size=[0.16, 0.16, 0.16]
    point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1]
    max_num_points=32
    max_voxels=(16000, 40000)

    test_layer = PillarLayer(voxel_size=voxel_size,
                            point_cloud_range=point_cloud_range,
                            max_voxels=max_voxels,
                            max_num_points=max_num_points)

    # pc = np.random.uniform(-2, 8, size=[1000, 4]).astype(np.float32)
    pc_tensor = torch.from_numpy(pc).to(device)

    voxels_th, indices_th, num_p_in_vx_th = test_layer([pc_tensor])

    # print(voxels_th[0].shape)
    # print(indices_th[0].shape)
    # print(num_p_in_vx_th[0].shape)

    # voxels_th = voxels_th.cpu().numpy()
    # indices_th = indices_th.cpu().numpy()
    # num_p_in_vx_th = num_p_in_vx_th.cpu().numpy()

    print(voxels_th[0][0])
    print(indices_th[0][0])
    print(num_p_in_vx_th[0][0])



tensor([[22.3920,  4.3760,  0.9810,  0.3800],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000

In [20]:
from spconv.utils import Point2VoxelCPU3d
from spconv.pytorch.utils import PointToVoxel, gather_features_by_pc_voxel_id


In [15]:
coors_batch = []

cur_list = torch.Tensor([[0,0,0],
                        [1,1,1]])

for i, cur_coors in enumerate(cur_list):
    coors_batch.append(F.pad(cur_coors, (1, 0), value=i))

In [16]:
coors_batch = torch.cat(coors_batch, dim=0)

[tensor([0., 0., 0., 0.]), tensor([1., 1., 1., 1.])]

In [17]:
coors_batch = torch.cat(coors_batch, dim=0)

In [18]:
coors_batch

tensor([0., 0., 0., 0., 1., 1., 1., 1.])